In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
gameData = pd.read_csv('games.csv', dayfirst=False, parse_dates=[3])
playsData = pd.read_csv('plays.csv')
playersData = pd.read_csv('players.csv')
pffData = pd.read_csv('PFFScoutingData.csv')
trackingData2018 = pd.read_csv("tracking2018.csv")
trackingData2019 = pd.read_csv("tracking2019.csv")
trackingData2020 = pd.read_csv("tracking2020.csv")

In [3]:
def downcast(dataFrame):
    startMemoryOptimization = dataFrame.memory_usage().sum() / 1024 ** 2
    print('Memory Usage\033[1;31;43m Before \033[0;0mOptimization of Dataframe is:\033[1m {:.2f} MB \033[0m'.format(startMemoryOptimization))
    floatColumns = [c for c in dataFrame if dataFrame[c].dtype in ['float64']]
    intColumns = [c for c in dataFrame if dataFrame[c].dtype in ['int64']]
    objectColumns = [c for c in dataFrame if dataFrame[c].dtype in ['object']]
    dataFrame[floatColumns] = dataFrame[floatColumns].astype(np.float32)
    dataFrame[intColumns] = dataFrame[intColumns].astype(np.int32)
    for column in dataFrame[objectColumns].columns:
        if column =='date':
                dataFrame['date'] = pd.to_datetime(dataFrame['date'],format='%Y-%m-%d')
                
    endMemoryOptimization = dataFrame.memory_usage().sum() / 1024 ** 2
    print('Memory Usage\033[1;31;43m After \033[0;0mOptimization is:\033[1m {:.2f} MB \033[0m'.format(endMemoryOptimization))
    print('\033[1m'+'Compressed by:\033[1;30;45m {:.2f} % \033[0;0m'.format(100*(startMemoryOptimization - endMemoryOptimization) / startMemoryOptimization))
    return dataFrame


print('\n' + '\033[1;30;42m' + ' Game Data Compression: ' + '\033[0m')
gameData = downcast(gameData)
print('\n' + '\033[1;30;42m' + ' Plays Data Compression: ' + '\033[0m')
playsData = downcast(playsData)
print('\n' + '\033[1;30;42m' + ' Playres Data Compression: ' + '\033[0m')
playersData = downcast(playersData)
print('\n' + '\033[1;30;42m' + ' Tracking Data 2018 Compression: ' + '\033[0m')
trackingData2018 = downcast(trackingData2018)
print('\n' + '\033[1;30;42m' + ' Tracking Data 2019 Compression: ' + '\033[0m')
trackingData2019 = downcast(trackingData2019)
print('\n' + '\033[1;30;42m' + ' Tracking Data 2020 Compression: ' + '\033[0m')
trackingData2020 = downcast(trackingData2020)


 Game Data Compression: 
Memory Usage Before Optimization of Dataframe is: 0.04 MB 
Memory Usage After Optimization is: 0.03 MB 
Compressed by: 21.36 % 

 Plays Data Compression: 
Memory Usage Before Optimization of Dataframe is: 3.81 MB 
Memory Usage After Optimization is: 2.67 MB 
Compressed by: 30.00 % 

 Playres Data Compression: 
Memory Usage Before Optimization of Dataframe is: 0.15 MB 
Memory Usage After Optimization is: 0.13 MB 
Compressed by: 14.27 % 

 Tracking Data 2018 Compression: 
Memory Usage Before Optimization of Dataframe is: 1754.70 MB 
Memory Usage After Optimization is: 1169.80 MB 
Compressed by: 33.33 % 

 Tracking Data 2019 Compression: 
Memory Usage Before Optimization of Dataframe is: 1671.42 MB 
Memory Usage After Optimization is: 1114.28 MB 
Compressed by: 33.33 % 

 Tracking Data 2020 Compression: 
Memory Usage Before Optimization of Dataframe is: 1623.46 MB 
Memory Usage After Optimization is: 1082.31 MB 
Compressed by: 33.33 % 


In [4]:
trackingFull = pd.concat([trackingData2018, trackingData2019, trackingData2020], axis=0)
trackingFull.reset_index(drop=True, inplace=True)

In [5]:
mergedGameAndPlaysData = gameData.merge(playsData, how='inner', indicator=True)
mergedGamePlaysAndTrackingFullData = mergedGameAndPlaysData.merge(trackingFull, how='inner', indicator='exists')

In [6]:
mergedGamePlaysAndTrackingFullData["Result"] = mergedGamePlaysAndTrackingFullData["specialTeamsPlayType"]+ " - " + mergedGamePlaysAndTrackingFullData["specialTeamsResult"].map(str)
mergedGamePlaysAndTrackingFullData.drop(['specialTeamsPlayType','specialTeamsResult'],axis=1, inplace=True)


In [7]:
mergedGamePlaysAndTrackingFullData.shape

(36769985, 48)

In [8]:
mergedGamePlaysAndTrackingFullData['Result'].unique()

array(['Kickoff - Touchback', 'Punt - Return',
       'Field Goal - Kick Attempt Good', 'Kickoff - Return',
       'Punt - Touchback', 'Punt - Fair Catch', 'Punt - Downed',
       'Extra Point - Kick Attempt Good', 'Punt - Muffed',
       'Extra Point - Kick Attempt No Good', 'Punt - Out of Bounds',
       'Field Goal - Kick Attempt No Good',
       'Punt - Non-Special Teams Result',
       'Field Goal - Blocked Kick Attempt',
       'Extra Point - Non-Special Teams Result', 'Kickoff - Muffed',
       'Punt - Blocked Punt', 'Extra Point - Blocked Kick Attempt',
       'Kickoff - Kickoff Team Recovery', 'Kickoff - Out of Bounds',
       'Field Goal - Non-Special Teams Result', 'Kickoff - Fair Catch',
       'Kickoff - Downed', 'Field Goal - Out of Bounds',
       'Field Goal - Downed'], dtype=object)

In [9]:
fieldGoalDataFrame = mergedGamePlaysAndTrackingFullData[['season', 'gameId','playId','possessionTeam','team','Result','kickerId',
                                                         'kickBlockerId','yardlineNumber','event','nflId','displayName','position']]

In [10]:
fieldgoalEvents = ['Field Goal - Kick Attempt Good','Field Goal - Kick Attempt No Good','Field Goal - Blocked Kick Attempt',
                   'Field Goal - Non-Special Teams Result','Field Goal - Out of Bounds','Field Goal - Downed']
fieldGoalEventsFrame = fieldGoalDataFrame.loc[fieldGoalDataFrame['Result'].isin(fieldgoalEvents)]

In [255]:
onlyFieldGoalOf2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]
onlyFieldGoalAtemptOf2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event.isin(['field_goal_attempt','field_goal_missed'])) &
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]
onlyFieldGoalMissedOf2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_missed') & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]
onlyFieldGoalBlockedOf2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_blocked') & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom1To19Yards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(1,19))
                                          ]
onlyFieldGoalsAttemptFrom1To19Yards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(1,19))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom20To29Yards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal')  &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(20,29))
                                          ]
onlyFieldGoalsAttemptFrom20To29Yards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(20,29))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom30To39Yards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(30,39))
                                          ]
onlyFieldGoalsAttemptFrom30To39Yards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(30,39))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom40To49Yards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(40,49))
                                          ]
onlyFieldGoalsAttemptFrom40To49Yards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(40,100))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom50PlusYards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(50,100))
                                          ]
onlyFieldGoalsAttemptFrom50PlusYards2018 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2018 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(50,100))
                                          ]

In [256]:

# Field Goals
onlyFieldGoalOf2018Series = onlyFieldGoalOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalOf2018Frame = pd.DataFrame(onlyFieldGoalOf2018Series)

onlyFieldGoalOf2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Field Goals'}, inplace=True)

cleanFieldGoalOf2018FrameSeries = \
onlyFieldGoalOf2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Field Goals'].sum().sort_values(ascending=False)
cleanFieldGoalOf2018FrameFrame = pd.DataFrame(cleanFieldGoalOf2018FrameSeries)
fieldGoals2018Ranking = cleanFieldGoalOf2018FrameFrame.reset_index()

###############################################################################################################################################################

# Field Goal Attempts
onlyFieldGoalAttemptsOf2018Series = onlyFieldGoalAtemptOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalAttemptsOf2018Frame = pd.DataFrame(onlyFieldGoalAttemptsOf2018Series)

onlyFieldGoalAttemptsOf2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'FG Attempts'}, inplace=True)

cleanFieldGoalAttemptsOf2018FrameSeries = \
onlyFieldGoalAttemptsOf2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'FG Attempts'].sum().sort_values(ascending=False)
cleanFieldGoalAttemptsOf2018FrameFrame = pd.DataFrame(cleanFieldGoalAttemptsOf2018FrameSeries)
fieldGoalsAttempts2018Ranking = cleanFieldGoalAttemptsOf2018FrameFrame.reset_index()

###############################################################################################################################################################

# Field Goal Missed
onlyFieldGoalMissedOf2018Series = onlyFieldGoalMissedOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalMissedOf2018Frame = pd.DataFrame(onlyFieldGoalMissedOf2018Series)

onlyFieldGoalMissedOf2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'FG Missed'}, inplace=True)

cleanFieldGoalMissedOf2018FrameSeries = \
onlyFieldGoalMissedOf2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'FG Missed'].sum().sort_values(ascending=False)
cleanFieldGoalMissedOf2018FrameFrame = pd.DataFrame(cleanFieldGoalMissedOf2018FrameSeries)
fieldGoalsMissed2018Ranking = cleanFieldGoalMissedOf2018FrameFrame.reset_index()

###############################################################################################################################################################

# Field Goal Blocked
onlyFieldGoalBlockedOf2018Series = onlyFieldGoalBlockedOf2018.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalBlockedOf2018Frame = pd.DataFrame(onlyFieldGoalBlockedOf2018Series)

onlyFieldGoalBlockedOf2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'FG Blocked'}, inplace=True)

cleanFieldGoalBlockedOf2018FrameSeries = \
onlyFieldGoalBlockedOf2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'FG Blocked'].sum().sort_values(ascending=False)
cleanFieldGoalBlockedOf2018FrameFrame = pd.DataFrame(cleanFieldGoalBlockedOf2018FrameSeries)
fieldGoalsBlocked2018Ranking = cleanFieldGoalBlockedOf2018FrameFrame.reset_index()


In [257]:

# Field Goals from 1 to 19 Yards
fieldGoalFrom1To19Yards2018Series = onlyFieldGoalsFrom1To19Yards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom1To19Yards2018Frame = pd.DataFrame(fieldGoalFrom1To19Yards2018Series)

fieldGoalFrom1To19Yards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '1-19 FG'}, inplace=True)

cleanFieldGoalFrom1To19Yards2018Series = \
fieldGoalFrom1To19Yards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '1-19 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom1To19Yards2018Frame = pd.DataFrame(cleanFieldGoalFrom1To19Yards2018Series)
fieldGoals2018RankingFor1To19Yards = cleanFieldGoalFrom1To19Yards2018Frame.reset_index()


# Field Goals Attempts from 1 to 19 Yards
onlyFieldGoalsAttemptFrom1To19Yards2018Series = onlyFieldGoalsAttemptFrom1To19Yards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom1To19Yards2018Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom1To19Yards2018Series)

onlyFieldGoalsAttemptFrom1To19Yards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '1-19 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom1To19Yards2018Series = \
onlyFieldGoalsAttemptFrom1To19Yards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '1-19 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom1To19Yards2018Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom1To19Yards2018Series)
fieldGoalsAttempts2018RankingFor1To19Yards = cleanFieldGoalsAttemptFrom1To19Yards2018Frame.reset_index()


###############################################################################################################################################################

# Field Goals from 20 to 29 Yards
fieldGoalFrom20To29Yards2018Series = onlyFieldGoalsFrom20To29Yards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom20To29Yards2018Frame = pd.DataFrame(fieldGoalFrom20To29Yards2018Series)

fieldGoalFrom20To29Yards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '20-29 FG'}, inplace=True)

cleanFieldGoalFrom20To29Yards2018Series = \
fieldGoalFrom20To29Yards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '20-29 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom20To29Yards2018Frame = pd.DataFrame(cleanFieldGoalFrom20To29Yards2018Series)
fieldGoals2018RankingFor20To29Yards = cleanFieldGoalFrom20To29Yards2018Frame.reset_index()


# Field Goals Attempts from 20 to 19 Yards
onlyFieldGoalsAttemptFrom20To29Yards2018Series = onlyFieldGoalsAttemptFrom20To29Yards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom20To29Yards2018Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom20To29Yards2018Series)

onlyFieldGoalsAttemptFrom20To29Yards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '20-29 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom20To29Yards2018Series = \
onlyFieldGoalsAttemptFrom20To29Yards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '20-29 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom20To29Yards2018Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom20To29Yards2018Series)
fieldGoalsAttempts2018RankingFor20To29Yards = cleanFieldGoalsAttemptFrom20To29Yards2018Frame.reset_index()


###############################################################################################################################################################


# Field Goals from 30 to 39 Yards
fieldGoalFrom30To39Yards2018Series = onlyFieldGoalsFrom30To39Yards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom30To39Yards2018Frame = pd.DataFrame(fieldGoalFrom30To39Yards2018Series)

fieldGoalFrom30To39Yards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '30-39 FG'}, inplace=True)

cleanFieldGoalFrom30To39Yards2018Series = \
fieldGoalFrom30To39Yards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '30-39 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom30To39Yards2018Frame = pd.DataFrame(cleanFieldGoalFrom30To39Yards2018Series)
fieldGoals2018RankingFor30To39Yards = cleanFieldGoalFrom30To39Yards2018Frame.reset_index()


# Field Goals Attempts from 30 to 39 Yards
onlyFieldGoalsAttemptFrom30To39Yards2018Series = onlyFieldGoalsAttemptFrom30To39Yards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom30To39Yards2018Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom30To39Yards2018Series)

onlyFieldGoalsAttemptFrom30To39Yards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '30-39 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom30To39Yards2018Series = \
onlyFieldGoalsAttemptFrom30To39Yards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '30-39 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom30To39Yards2018Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom30To39Yards2018Series)
fieldGoalsAttempts2018RankingFor30To39Yards = cleanFieldGoalsAttemptFrom30To39Yards2018Frame.reset_index()


###############################################################################################################################################################


# Field Goals from 40 to 49 Yards
fieldGoalFrom40To49Yards2018Series = onlyFieldGoalsFrom40To49Yards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom40To49Yards2018Frame = pd.DataFrame(fieldGoalFrom40To49Yards2018Series)

fieldGoalFrom40To49Yards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '40-49 FG'}, inplace=True)

cleanFieldGoalFrom40To49Yards2018Series = \
fieldGoalFrom40To49Yards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '40-49 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom40To49Yards2018Frame = pd.DataFrame(cleanFieldGoalFrom40To49Yards2018Series)
fieldGoals2018RankingFor40To49Yards = cleanFieldGoalFrom40To49Yards2018Frame.reset_index()


# Field Goals Attempts from 40 to 49 Yards
onlyFieldGoalsAttemptFrom40To49Yards2018Series = onlyFieldGoalsAttemptFrom40To49Yards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom40To49Yards2018Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom40To49Yards2018Series)

onlyFieldGoalsAttemptFrom40To49Yards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '40-49 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom40To49Yards2018Series = \
onlyFieldGoalsAttemptFrom40To49Yards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '40-49 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom40To49Yards2018Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom40To49Yards2018Series)
fieldGoalsAttempts2018RankingFor40To49Yards = cleanFieldGoalsAttemptFrom40To49Yards2018Frame.reset_index()


###############################################################################################################################################################

# Field Goals from 50+ Yards
fieldGoalFrom50PlusYards2018Series = onlyFieldGoalsFrom50PlusYards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom50PlusYards2018Frame = pd.DataFrame(fieldGoalFrom50PlusYards2018Series)

fieldGoalFrom50PlusYards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '50+ FG'}, inplace=True)

cleanFieldGoalFrom50PlusYards2018Series = \
fieldGoalFrom50PlusYards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '50+ FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom50PlusYards2018Frame = pd.DataFrame(cleanFieldGoalFrom50PlusYards2018Series)
fieldGoals2018RankingFor50PlusYards = cleanFieldGoalFrom50PlusYards2018Frame.reset_index()


# Field Goals Attempts from 50+ Yards
onlyFieldGoalsAttemptFrom50PlusYards2018Series = onlyFieldGoalsAttemptFrom50PlusYards2018.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom50PlusYards2018Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom50PlusYards2018Series)

onlyFieldGoalsAttemptFrom50PlusYards2018Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '50+ FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom50PlusYards2018Series = \
onlyFieldGoalsAttemptFrom50PlusYards2018Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '50+ FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom50PlusYards2018Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom50PlusYards2018Series)
fieldGoalsAttempts2018RankingFor50PlusYards = cleanFieldGoalsAttemptFrom50PlusYards2018Frame.reset_index()

In [258]:


###############################################################################################################################################################
mergeFiledGoalAndAttemptsFrom1To19Yards = pd.merge(fieldGoals2018RankingFor1To19Yards, 
                                                   fieldGoalsAttempts2018RankingFor1To19Yards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom1To19Yards.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom1To19Yards['1-19 FG'] = mergeFiledGoalAndAttemptsFrom1To19Yards['1-19 FG'].astype(int)

mergeFiledGoalAndAttemptsFrom1To19Yards["1 - 19: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom1To19Yards["1-19 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom1To19Yards["1-19 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom1To19Yards.drop(['1-19 FG','1-19 FGA'],axis=1, inplace=True)


###############################################################################################################################################################


mergeFiledGoalAndAttemptsFrom20To29Yards = pd.merge(fieldGoals2018RankingFor20To29Yards, 
                                                   fieldGoalsAttempts2018RankingFor20To29Yards,
                                                   how='right')

mergeFiledGoalAndAttemptsFrom20To29Yards.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom20To29Yards['20-29 FG'] = mergeFiledGoalAndAttemptsFrom20To29Yards['20-29 FG'].astype(int)



mergeFiledGoalAndAttemptsFrom20To29Yards["20 - 29: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom20To29Yards["20-29 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom20To29Yards["20-29 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom20To29Yards.drop(['20-29 FG','20-29 FGA'],axis=1, inplace=True)


###############################################################################################################################################################


mergeFiledGoalAndAttemptsFrom30To39Yards = pd.merge(fieldGoals2018RankingFor30To39Yards, 
                                                   fieldGoalsAttempts2018RankingFor30To39Yards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom30To39Yards.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom30To39Yards['30-39 FG'] = mergeFiledGoalAndAttemptsFrom30To39Yards['30-39 FG'].astype(int)



mergeFiledGoalAndAttemptsFrom30To39Yards["30 - 39: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom30To39Yards["30-39 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom30To39Yards["30-39 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom30To39Yards.drop(['30-39 FG','30-39 FGA'],axis=1, inplace=True)


###############################################################################################################################################################

mergeFiledGoalAndAttemptsFrom40To49Yards = pd.merge(fieldGoals2018RankingFor40To49Yards, 
                                                   fieldGoalsAttempts2018RankingFor40To49Yards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom40To49Yards.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom40To49Yards['40-49 FG'] = mergeFiledGoalAndAttemptsFrom40To49Yards['40-49 FG'].astype(int)



mergeFiledGoalAndAttemptsFrom40To49Yards["40 - 49: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom40To49Yards["40-49 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom40To49Yards["40-49 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom40To49Yards.drop(['40-49 FG','40-49 FGA'],axis=1, inplace=True)


###############################################################################################################################################################

mergeFiledGoalAndAttemptsFrom50PlusYards = pd.merge(fieldGoals2018RankingFor50PlusYards, 
                                                   fieldGoalsAttempts2018RankingFor50PlusYards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom50PlusYards.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom50PlusYards['50+ FG'] = mergeFiledGoalAndAttemptsFrom50PlusYards['50+ FG'].astype(int)



mergeFiledGoalAndAttemptsFrom50PlusYards["50+: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom50PlusYards["50+ FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom50PlusYards["50+ FGA"].map(str)
mergeFiledGoalAndAttemptsFrom50PlusYards.drop(['50+ FG','50+ FGA'],axis=1, inplace=True)



In [259]:
mergeFiledGoalAndAttempts = pd.merge(fieldGoals2018Ranking, 
                                     fieldGoalsAttempts2018Ranking['FG Attempts'], 
                                     left_index=True, 
                                     right_index=True, 
                                     how='outer'
                                    )

###############################################################################################################################################################

mergeFiledGoalAndAttempts['FG Accuracy %'] = (mergeFiledGoalAndAttempts['Field Goals'] / mergeFiledGoalAndAttempts['FG Attempts']).round(3) * 100

###############################################################################################################################################################

mergeFiledGoalAndAttempts.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttempts['Field Goals'] = mergeFiledGoalAndAttempts['Field Goals'].astype(int)


In [260]:

addFGFor1To19Yards = pd.merge(mergeFiledGoalAndAttempts, 
                              mergeFiledGoalAndAttemptsFrom1To19Yards, 
                              how='left'
                             )
###############################################################################################################################################################

addFGFor20To29Yards = pd.merge(addFGFor1To19Yards, 
                               mergeFiledGoalAndAttemptsFrom20To29Yards, 
                               how='left'
                              )
###############################################################################################################################################################

addFGFor30To39Yards = pd.merge(addFGFor20To29Yards, 
                               mergeFiledGoalAndAttemptsFrom30To39Yards, 
                               how='left'
                              )

###############################################################################################################################################################

addFGFor40To49Yards = pd.merge(addFGFor30To39Yards, 
                               mergeFiledGoalAndAttemptsFrom40To49Yards, 
                               how='left'
                              )

###############################################################################################################################################################

addFGFor40To49Yards.replace(np.nan,'0 / 0', inplace = True)


In [261]:

mergeFiledGoalMissed = pd.merge(addFGFor40To49Yards, 
                                fieldGoalsMissed2018Ranking, 
                                how='outer'
                               )

mergeFiledGoalMissed.replace(np.nan, 0, inplace=True)


###############################################################################################################################################################

mergeFieldGoalBlocked = pd.merge(mergeFiledGoalMissed,
                                 fieldGoalsBlocked2018Ranking,
                                 how='outer'
                                )

initial2018FieldGoalStat = mergeFieldGoalBlocked.replace(np.nan, 0)

###############################################################################################################################################################

initial2018FieldGoalStat['Season'] = initial2018FieldGoalStat['Season'].astype(int)
initial2018FieldGoalStat['Field Goals'] = initial2018FieldGoalStat['Field Goals'].astype(int)
initial2018FieldGoalStat['FG Attempts'] = initial2018FieldGoalStat['FG Attempts'].astype(int)
initial2018FieldGoalStat['FG Missed'] = initial2018FieldGoalStat['FG Missed'].astype(int)
initial2018FieldGoalStat['FG Blocked'] = initial2018FieldGoalStat['FG Blocked'].astype(int)

In [262]:

final2018FieldGoalStat = initial2018FieldGoalStat[(initial2018FieldGoalStat.Season != 0)]

FieldGoal2018 = final2018FieldGoalStat.rename(
    columns={'Field Goals': 'FG Made', '1 - 19: FGM / FGA': '1-19 yds: FGM/FGA', '20 - 29: FGM / FGA': '20-29 yds: FGM/FGA', '30 - 39: FGM / FGA': '30-39 yds: FGM/FGA',
             '40 - 49: FGM / FGA': '40-49 yds: FGM/FGA'})


In [263]:
FieldGoal2018

,Season,Player Name,Team,Position,FG Made,FG Attempts,FG Accuracy %,1-19 yds: FGM/FGA,20-29 yds: FGM/FGA,30-39 yds: FGM/FGA,40-49 yds: FGM/FGA,FG Missed,FG Blocked
0,2018,Ka'imi Fairbairn,HOU,K,36,46,78.3,20 / 20,11 / 13,5 / 7,0 / 1,5,0
1,2018,Justin Tucker,BAL,K,35,41,85.4,19 / 19,10 / 11,6 / 7,0 / 1,0,2
2,2018,Jason Myers,NYJ,K,33,41,80.5,13 / 14,11 / 12,9 / 10,0 / 0,3,0
3,2018,Robbie Gould,SF,K,30,39,76.9,18 / 18,10 / 11,2 / 2,0 / 0,1,0
4,2018,Aldrick Rosas,NYG,K,30,38,78.9,19 / 19,6 / 6,5 / 6,0 / 0,1,0
5,2018,Mason Crosby,GB,K,29,37,78.4,10 / 10,12 / 16,7 / 9,0 / 0,6,0
6,2018,Brett Maher,DAL,K,28,37,75.7,15 / 16,7 / 11,5 / 7,1 / 1,6,1
7,2018,Stephen Gostkowski,NE,K,27,36,75.0,18 / 18,6 / 7,3 / 7,0 / 0,5,0
8,2018,Wil Lutz,NO,K,26,34,76.5,11 / 11,12 / 12,3 / 4,0 / 0,0,1
9,2018,Jake Elliott,PHI,K,26,34,76.5,17 / 18,7 / 8,2 / 5,0 / 0,5,0


In [264]:

onlyFieldGoalOf2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]
onlyFieldGoalAtemptOf2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event.isin(['field_goal_attempt','field_goal_missed'])) & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]
onlyFieldGoalMissedOf2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_missed') & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]
onlyFieldGoalBlockedOf2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_blocked') & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom1To19Yards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(1,19))
                                          ]
onlyFieldGoalsAttemptFrom1To19Yards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(1,19))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom20To29Yards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal')  &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(20,29))
                                          ]
onlyFieldGoalsAttemptFrom20To29Yards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(20,29))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom30To39Yards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(30,39))
                                          ]
onlyFieldGoalsAttemptFrom30To39Yards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(30,39))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom40To49Yards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(40,100))
                                          ]
onlyFieldGoalsAttemptFrom40To49Yards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(40,49))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom50PlusYards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(50,100))
                                          ]
onlyFieldGoalsAttemptFrom50PlusYards2019 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2019 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(50,100))
                                          ]


In [265]:

# Field Goals
onlyFieldGoalOf2019Series = onlyFieldGoalOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalOf2019Frame = pd.DataFrame(onlyFieldGoalOf2019Series)

onlyFieldGoalOf2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Field Goals'}, inplace=True)

cleanFieldGoalOf2019FrameSeries = \
onlyFieldGoalOf2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Field Goals'].sum().sort_values(ascending=False)
cleanFieldGoalOf2019FrameFrame = pd.DataFrame(cleanFieldGoalOf2019FrameSeries)
fieldGoals2019Ranking = cleanFieldGoalOf2019FrameFrame.reset_index()

###############################################################################################################################################################

# Field Goal Attempts
onlyFieldGoalAttemptsOf2019Series = onlyFieldGoalAtemptOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalAttemptsOf2019Frame = pd.DataFrame(onlyFieldGoalAttemptsOf2019Series)

onlyFieldGoalAttemptsOf2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'FG Attempts'}, inplace=True)

cleanFieldGoalAttemptsOf2019FrameSeries = \
onlyFieldGoalAttemptsOf2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'FG Attempts'].sum().sort_values(ascending=False)
cleanFieldGoalAttemptsOf2019FrameFrame = pd.DataFrame(cleanFieldGoalAttemptsOf2019FrameSeries)
fieldGoalsAttempts2019Ranking = cleanFieldGoalAttemptsOf2019FrameFrame.reset_index()

###############################################################################################################################################################

# Field Goal Missed
onlyFieldGoalMissedOf2019Series = onlyFieldGoalMissedOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalMissedOf2019Frame = pd.DataFrame(onlyFieldGoalMissedOf2019Series)

onlyFieldGoalMissedOf2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'FG Missed'}, inplace=True)

cleanFieldGoalMissedOf2019FrameSeries = \
onlyFieldGoalMissedOf2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'FG Missed'].sum().sort_values(ascending=False)
cleanFieldGoalMissedOf2019FrameFrame = pd.DataFrame(cleanFieldGoalMissedOf2019FrameSeries)
fieldGoalsMissed2019Ranking = cleanFieldGoalMissedOf2019FrameFrame.reset_index()

###############################################################################################################################################################

# Field Goal Blocked
onlyFieldGoalBlockedOf2019Series = onlyFieldGoalBlockedOf2019.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalBlockedOf2019Frame = pd.DataFrame(onlyFieldGoalBlockedOf2019Series)

onlyFieldGoalBlockedOf2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'FG Blocked'}, inplace=True)

cleanFieldGoalBlockedOf2019FrameSeries = \
onlyFieldGoalBlockedOf2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'FG Blocked'].sum().sort_values(ascending=False)
cleanFieldGoalBlockedOf2019FrameFrame = pd.DataFrame(cleanFieldGoalBlockedOf2019FrameSeries)
fieldGoalsBlocked2019Ranking = cleanFieldGoalBlockedOf2019FrameFrame.reset_index()


In [266]:

# Field Goals from 1 to 19 Yards
fieldGoalFrom1To19Yards2019Series = onlyFieldGoalsFrom1To19Yards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom1To19Yards2019Frame = pd.DataFrame(fieldGoalFrom1To19Yards2019Series)

fieldGoalFrom1To19Yards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '1-19 FG'}, inplace=True)

cleanFieldGoalFrom1To19Yards2019Series = \
fieldGoalFrom1To19Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '1-19 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom1To19Yards2019Frame = pd.DataFrame(cleanFieldGoalFrom1To19Yards2019Series)
fieldGoals2019RankingFor1To19Yards = cleanFieldGoalFrom1To19Yards2019Frame.reset_index()


# Field Goals Attempts from 1 to 19 Yards
onlyFieldGoalsAttemptFrom1To19Yards2019Series = onlyFieldGoalsAttemptFrom1To19Yards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom1To19Yards2019Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom1To19Yards2019Series)

onlyFieldGoalsAttemptFrom1To19Yards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '1-19 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom1To19Yards2019Series = \
onlyFieldGoalsAttemptFrom1To19Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '1-19 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom1To19Yards2019Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom1To19Yards2019Series)
fieldGoalsAttempts2019RankingFor1To19Yards = cleanFieldGoalsAttemptFrom1To19Yards2019Frame.reset_index()


###############################################################################################################################################################

# Field Goals from 20 to 29 Yards
fieldGoalFrom20To29Yards2019Series = onlyFieldGoalsFrom20To29Yards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom20To29Yards2019Frame = pd.DataFrame(fieldGoalFrom20To29Yards2019Series)

fieldGoalFrom20To29Yards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '20-29 FG'}, inplace=True)

cleanFieldGoalFrom20To29Yards2019Series = \
fieldGoalFrom20To29Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '20-29 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom20To29Yards2019Frame = pd.DataFrame(cleanFieldGoalFrom20To29Yards2019Series)
fieldGoals2019RankingFor20To29Yards = cleanFieldGoalFrom20To29Yards2019Frame.reset_index()


# Field Goals Attempts from 20 to 19 Yards
onlyFieldGoalsAttemptFrom20To29Yards2019Series = onlyFieldGoalsAttemptFrom20To29Yards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom20To29Yards2019Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom20To29Yards2019Series)

onlyFieldGoalsAttemptFrom20To29Yards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '20-29 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom20To29Yards2019Series = \
onlyFieldGoalsAttemptFrom20To29Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '20-29 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom20To29Yards2019Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom20To29Yards2019Series)
fieldGoalsAttempts2019RankingFor20To29Yards = cleanFieldGoalsAttemptFrom20To29Yards2019Frame.reset_index()


###############################################################################################################################################################


# Field Goals from 30 to 39 Yards
fieldGoalFrom30To39Yards2019Series = onlyFieldGoalsFrom30To39Yards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom30To39Yards2019Frame = pd.DataFrame(fieldGoalFrom30To39Yards2019Series)

fieldGoalFrom30To39Yards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '30-39 FG'}, inplace=True)

cleanFieldGoalFrom30To39Yards2019Series = \
fieldGoalFrom30To39Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '30-39 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom30To39Yards2019Frame = pd.DataFrame(cleanFieldGoalFrom30To39Yards2019Series)
fieldGoals2019RankingFor30To39Yards = cleanFieldGoalFrom30To39Yards2019Frame.reset_index()


# Field Goals Attempts from 30 to 39 Yards
onlyFieldGoalsAttemptFrom30To39Yards2019Series = onlyFieldGoalsAttemptFrom30To39Yards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom30To39Yards2019Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom30To39Yards2019Series)

onlyFieldGoalsAttemptFrom30To39Yards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '30-39 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom30To39Yards2019Series = \
onlyFieldGoalsAttemptFrom30To39Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '30-39 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom30To39Yards2019Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom30To39Yards2019Series)
fieldGoalsAttempts2019RankingFor30To39Yards = cleanFieldGoalsAttemptFrom30To39Yards2019Frame.reset_index()


###############################################################################################################################################################


# Field Goals from 40 to 49 Yards
fieldGoalFrom40To49Yards2019Series = onlyFieldGoalsFrom40To49Yards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom40To49Yards2019Frame = pd.DataFrame(fieldGoalFrom40To49Yards2019Series)

fieldGoalFrom40To49Yards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '40-49 FG'}, inplace=True)

cleanFieldGoalFrom40To49Yards2019Series = \
fieldGoalFrom40To49Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '40-49 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom40To49Yards2019Frame = pd.DataFrame(cleanFieldGoalFrom40To49Yards2019Series)
fieldGoals2019RankingFor40To49Yards = cleanFieldGoalFrom40To49Yards2019Frame.reset_index()


# Field Goals Attempts from 40 to 49 Yards
onlyFieldGoalsAttemptFrom40To49Yards2019Series = onlyFieldGoalsAttemptFrom40To49Yards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom40To49Yards2019Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom40To49Yards2019Series)

onlyFieldGoalsAttemptFrom40To49Yards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '40-49 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom40To49Yards2019Series = \
onlyFieldGoalsAttemptFrom40To49Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '40-49 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom40To49Yards2019Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom40To49Yards2019Series)
fieldGoalsAttempts2019RankingFor40To49Yards = cleanFieldGoalsAttemptFrom40To49Yards2019Frame.reset_index()


###############################################################################################################################################################

# Field Goals from 50+ Yards
fieldGoalFrom50PlusYards2019Series = onlyFieldGoalsFrom50PlusYards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom50PlusYards2019Frame = pd.DataFrame(fieldGoalFrom50PlusYards2019Series)

fieldGoalFrom50PlusYards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '50+ FG'}, inplace=True)

cleanFieldGoalFrom50PlusYards2019Series = \
fieldGoalFrom50PlusYards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '50+ FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom50PlusYards2019Frame = pd.DataFrame(cleanFieldGoalFrom50PlusYards2019Series)
fieldGoals2019RankingFor50PlusYards = cleanFieldGoalFrom50PlusYards2019Frame.reset_index()


# Field Goals Attempts from 50+ Yards
onlyFieldGoalsAttemptFrom50PlusYards2019Series = onlyFieldGoalsAttemptFrom50PlusYards2019.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom50PlusYards2019Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom50PlusYards2019Series)

onlyFieldGoalsAttemptFrom50PlusYards2019Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '50+ FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom50PlusYards2019Series = \
onlyFieldGoalsAttemptFrom50PlusYards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '50+ FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom50PlusYards2019Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom50PlusYards2019Series)
fieldGoalsAttempts2019RankingFor50PlusYards = cleanFieldGoalsAttemptFrom50PlusYards2019Frame.reset_index()

In [267]:


###############################################################################################################################################################
mergeFiledGoalAndAttemptsFrom1To19Yards2019 = pd.merge(fieldGoals2019RankingFor1To19Yards, 
                                                   fieldGoalsAttempts2019RankingFor1To19Yards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom1To19Yards2019.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom1To19Yards2019['1-19 FG'] = mergeFiledGoalAndAttemptsFrom1To19Yards2019['1-19 FG'].astype(int)

mergeFiledGoalAndAttemptsFrom1To19Yards2019["1 - 19: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom1To19Yards2019["1-19 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom1To19Yards2019["1-19 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom1To19Yards2019.drop(['1-19 FG','1-19 FGA'],axis=1, inplace=True)


###############################################################################################################################################################


mergeFiledGoalAndAttemptsFrom20To29Yards2019 = pd.merge(fieldGoals2019RankingFor20To29Yards, 
                                                   fieldGoalsAttempts2019RankingFor20To29Yards,
                                                   how='right')

mergeFiledGoalAndAttemptsFrom20To29Yards2019.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom20To29Yards2019['20-29 FG'] = mergeFiledGoalAndAttemptsFrom20To29Yards2019['20-29 FG'].astype(int)



mergeFiledGoalAndAttemptsFrom20To29Yards2019["20 - 29: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom20To29Yards2019["20-29 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom20To29Yards2019["20-29 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom20To29Yards2019.drop(['20-29 FG','20-29 FGA'],axis=1, inplace=True)


###############################################################################################################################################################


mergeFiledGoalAndAttemptsFrom30To39Yards2019 = pd.merge(fieldGoals2019RankingFor30To39Yards, 
                                                   fieldGoalsAttempts2019RankingFor30To39Yards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom30To39Yards2019.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom30To39Yards2019['30-39 FG'] = mergeFiledGoalAndAttemptsFrom30To39Yards2019['30-39 FG'].astype(int)



mergeFiledGoalAndAttemptsFrom30To39Yards2019["30 - 39: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom30To39Yards2019["30-39 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom30To39Yards2019["30-39 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom30To39Yards2019.drop(['30-39 FG','30-39 FGA'],axis=1, inplace=True)


###############################################################################################################################################################

mergeFiledGoalAndAttemptsFrom40To49Yards2019 = pd.merge(fieldGoals2019RankingFor40To49Yards, 
                                                   fieldGoalsAttempts2019RankingFor40To49Yards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom40To49Yards2019.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom40To49Yards2019['40-49 FG'] = mergeFiledGoalAndAttemptsFrom40To49Yards2019['40-49 FG'].astype(int)



mergeFiledGoalAndAttemptsFrom40To49Yards2019["40 - 49: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom40To49Yards2019["40-49 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom40To49Yards2019["40-49 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom40To49Yards2019.drop(['40-49 FG','40-49 FGA'],axis=1, inplace=True)


###############################################################################################################################################################

mergeFiledGoalAndAttemptsFrom50PlusYards2019 = pd.merge(fieldGoals2019RankingFor50PlusYards, 
                                                   fieldGoalsAttempts2019RankingFor50PlusYards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom50PlusYards2019.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom50PlusYards2019['50+ FG'] = mergeFiledGoalAndAttemptsFrom50PlusYards2019['50+ FG'].astype(int)



mergeFiledGoalAndAttemptsFrom50PlusYards2019["50+: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom50PlusYards2019["50+ FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom50PlusYards2019["50+ FGA"].map(str)
mergeFiledGoalAndAttemptsFrom50PlusYards2019.drop(['50+ FG','50+ FGA'],axis=1, inplace=True)



In [268]:
mergeFiledGoalAndAttempts2019 = pd.merge(fieldGoals2019Ranking, 
                                     fieldGoalsAttempts2019Ranking['FG Attempts'], 
                                     left_index=True, 
                                     right_index=True, 
                                     how='outer'
                                    )

###############################################################################################################################################################

mergeFiledGoalAndAttempts2019['FG Accuracy %'] = (mergeFiledGoalAndAttempts2019['Field Goals'] / mergeFiledGoalAndAttempts2019['FG Attempts']).round(3) * 100

###############################################################################################################################################################

mergeFiledGoalAndAttempts2019.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttempts2019['Field Goals'] = mergeFiledGoalAndAttempts2019['Field Goals'].astype(int)


In [269]:

addFGFor1To19Yards2019 = pd.merge(mergeFiledGoalAndAttempts2019, 
                              mergeFiledGoalAndAttemptsFrom1To19Yards2019, 
                              how='left'
                             )
###############################################################################################################################################################

addFGFor20To29Yards2019 = pd.merge(addFGFor1To19Yards2019, 
                               mergeFiledGoalAndAttemptsFrom20To29Yards2019, 
                               how='left'
                              )
###############################################################################################################################################################

addFGFor30To39Yards2019 = pd.merge(addFGFor20To29Yards2019, 
                               mergeFiledGoalAndAttemptsFrom30To39Yards2019, 
                               how='left'
                              )

###############################################################################################################################################################

addFGFor40To49Yards2019 = pd.merge(addFGFor30To39Yards2019, 
                               mergeFiledGoalAndAttemptsFrom40To49Yards2019, 
                               how='left'
                              )

###############################################################################################################################################################

addFGFor40To49Yards2019.replace(np.nan,'0 / 0', inplace = True)


In [270]:

mergeFiledGoalMissed2019 = pd.merge(addFGFor40To49Yards2019, 
                                fieldGoalsMissed2019Ranking, 
                                how='outer'
                               )

mergeFiledGoalMissed2019.replace(np.nan, 0, inplace=True)


###############################################################################################################################################################

mergeFieldGoalBlocked2019 = pd.merge(mergeFiledGoalMissed2019,
                                 fieldGoalsBlocked2019Ranking,
                                 how='outer'
                                )

initial2019FieldGoalStat = mergeFieldGoalBlocked2019.replace(np.nan, 0)

###############################################################################################################################################################

initial2019FieldGoalStat['Season'] = initial2019FieldGoalStat['Season'].astype(int)
initial2019FieldGoalStat['Field Goals'] = initial2019FieldGoalStat['Field Goals'].astype(int)
initial2019FieldGoalStat['FG Attempts'] = initial2019FieldGoalStat['FG Attempts'].astype(int)
initial2019FieldGoalStat['FG Missed'] = initial2019FieldGoalStat['FG Missed'].astype(int)
initial2019FieldGoalStat['FG Blocked'] = initial2019FieldGoalStat['FG Blocked'].astype(int)

In [271]:

final2019FieldGoalStat = initial2019FieldGoalStat[(initial2019FieldGoalStat.Season != 0)]

FieldGoal2019 = final2019FieldGoalStat.rename(
    columns={'Field Goals': 'FG Made', '1 - 19: FGM / FGA': '1-19 yds: FGM/FGA', '20 - 29: FGM / FGA': '20-29 yds: FGM/FGA', '30 - 39: FGM / FGA': '30-39 yds: FGM/FGA',
             '40 - 49: FGM / FGA': '40-49 yds: FGM/FGA'})


In [272]:
FieldGoal2019

,Season,Player Name,Team,Position,FG Made,FG Attempts,FG Accuracy %,1-19 yds: FGM/FGA,20-29 yds: FGM/FGA,30-39 yds: FGM/FGA,40-49 yds: FGM/FGA,FG Missed,FG Blocked
0,2019,Harrison Butker,KC,K,32,38,84.2,16 / 17,13 / 13,3 / 5,0 / 0,2,1
1,2019,Josh Lambo,JAX,K,30,37,81.1,18 / 18,7 / 8,5 / 5,0 / 0,1,0
2,2019,Wil Lutz,NO,K,28,35,80.0,15 / 15,10 / 13,2 / 3,1 / 1,3,0
3,2019,Justin Tucker,BAL,K,28,35,80.0,15 / 15,9 / 10,4 / 4,0 / 0,1,0
4,2019,Zane Gonzalez,ARI,K,28,35,80.0,21 / 22,4 / 5,3 / 4,0 / 0,2,1
5,2019,Brandon McManus,DEN,K,28,35,80.0,13 / 13,9 / 10,6 / 7,0 / 1,3,0
6,2019,Randy Bullock,CIN,K,26,34,76.5,13 / 13,10 / 11,3 / 6,0 / 0,4,0
7,2019,Dan Bailey,MIN,K,25,34,73.5,13 / 13,8 / 10,4 / 4,0 / 0,2,0
8,2019,Matt Gay,TB,K,25,34,73.5,10 / 12,9 / 10,5 / 9,1 / 1,6,1
9,2019,Chris Boswell,PIT,K,24,33,72.7,15 / 17,6 / 7,3 / 4,0 / 0,2,0


In [273]:

onlyFieldGoalOf2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]
onlyFieldGoalAtemptOf2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event.isin(['field_goal_attempt','field_goal_missed'])) & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]
onlyFieldGoalMissedOf2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_missed') & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]
onlyFieldGoalBlockedOf2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_blocked') & 
                                           (fieldGoalEventsFrame.position == 'K') 
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom1To19Yards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(1,19))
                                          ]
onlyFieldGoalsAttemptFrom1To19Yards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(1,19))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom20To29Yards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal')  &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(20,29))
                                          ]
onlyFieldGoalsAttemptFrom20To29Yards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(20,29))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom30To39Yards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020 ) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(30,39))
                                          ]
onlyFieldGoalsAttemptFrom30To39Yards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(30,39))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom40To49Yards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(40,49))
                                          ]
onlyFieldGoalsAttemptFrom40To49Yards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(40,100))
                                          ]

###############################################################################################################################################################


onlyFieldGoalsFrom50PlusYards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event == 'field_goal') &
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(50,100))
                                          ]
onlyFieldGoalsAttemptFrom50PlusYards2020 = fieldGoalEventsFrame[(fieldGoalEventsFrame.season == 2020) & 
                                           (fieldGoalEventsFrame.event == 'field_goal_attempt') & 
                                           (fieldGoalEventsFrame.position == 'K') &
                                           (fieldGoalEventsFrame.yardlineNumber.between(50,100))
                                          ]


In [274]:

# Field Goals
onlyFieldGoalOf2020Series = onlyFieldGoalOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalOf2020Frame = pd.DataFrame(onlyFieldGoalOf2020Series)

onlyFieldGoalOf2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'Field Goals'}, inplace=True)

cleanFieldGoalOf2020FrameSeries = \
onlyFieldGoalOf2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'Field Goals'].sum().sort_values(ascending=False)
cleanFieldGoalOf2020FrameFrame = pd.DataFrame(cleanFieldGoalOf2020FrameSeries)
fieldGoals2020Ranking = cleanFieldGoalOf2020FrameFrame.reset_index()

###############################################################################################################################################################

# Field Goal Attempts
onlyFieldGoalAttemptsOf2020Series = onlyFieldGoalAtemptOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalAttemptsOf2020Frame = pd.DataFrame(onlyFieldGoalAttemptsOf2020Series)

onlyFieldGoalAttemptsOf2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'FG Attempts'}, inplace=True)

cleanFieldGoalAttemptsOf2020FrameSeries = \
onlyFieldGoalAttemptsOf2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'FG Attempts'].sum().sort_values(ascending=False)
cleanFieldGoalAttemptsOf2020FrameFrame = pd.DataFrame(cleanFieldGoalAttemptsOf2020FrameSeries)
fieldGoalsAttempts2020Ranking = cleanFieldGoalAttemptsOf2020FrameFrame.reset_index()

###############################################################################################################################################################

# Field Goal Missed
onlyFieldGoalMissedOf2020Series = onlyFieldGoalMissedOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalMissedOf2020Frame = pd.DataFrame(onlyFieldGoalMissedOf2020Series)

onlyFieldGoalMissedOf2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'FG Missed'}, inplace=True)

cleanFieldGoalMissedOf2020FrameSeries = \
onlyFieldGoalMissedOf2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'FG Missed'].sum().sort_values(ascending=False)
cleanFieldGoalMissedOf2020FrameFrame = pd.DataFrame(cleanFieldGoalMissedOf2020FrameSeries)
fieldGoalsMissed2020Ranking = cleanFieldGoalMissedOf2020FrameFrame.reset_index()

###############################################################################################################################################################

# Field Goal Blocked
onlyFieldGoalBlockedOf2020Series = onlyFieldGoalBlockedOf2020.groupby(
    ['season', 'possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalBlockedOf2020Frame = pd.DataFrame(onlyFieldGoalBlockedOf2020Series)

onlyFieldGoalBlockedOf2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': 'FG Blocked'}, inplace=True)

cleanFieldGoalBlockedOf2020FrameSeries = \
onlyFieldGoalBlockedOf2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    'FG Blocked'].sum().sort_values(ascending=False)
cleanFieldGoalBlockedOf2020FrameFrame = pd.DataFrame(cleanFieldGoalBlockedOf2020FrameSeries)
fieldGoalsBlocked2020Ranking = cleanFieldGoalBlockedOf2020FrameFrame.reset_index()


In [275]:

# Field Goals from 1 to 19 Yards
fieldGoalFrom1To19Yards2020Series = onlyFieldGoalsFrom1To19Yards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom1To19Yards2020Frame = pd.DataFrame(fieldGoalFrom1To19Yards2020Series)

fieldGoalFrom1To19Yards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '1-19 FG'}, inplace=True)

cleanFieldGoalFrom1To19Yards2020Series = \
fieldGoalFrom1To19Yards2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '1-19 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom1To19Yards2020Frame = pd.DataFrame(cleanFieldGoalFrom1To19Yards2020Series)
fieldGoals2020RankingFor1To19Yards = cleanFieldGoalFrom1To19Yards2020Frame.reset_index()


# Field Goals Attempts from 1 to 19 Yards
onlyFieldGoalsAttemptFrom1To19Yards2020Series = onlyFieldGoalsAttemptFrom1To19Yards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom1To19Yards2020Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom1To19Yards2020Series)

onlyFieldGoalsAttemptFrom1To19Yards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '1-19 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom1To19Yards2020Series = \
onlyFieldGoalsAttemptFrom1To19Yards2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '1-19 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom1To19Yards2020Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom1To19Yards2020Series)
fieldGoalsAttempts2020RankingFor1To19Yards = cleanFieldGoalsAttemptFrom1To19Yards2020Frame.reset_index()


###############################################################################################################################################################

# Field Goals from 20 to 29 Yards
fieldGoalFrom20To29Yards2020Series = onlyFieldGoalsFrom20To29Yards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom20To29Yards2020Frame = pd.DataFrame(fieldGoalFrom20To29Yards2020Series)

fieldGoalFrom20To29Yards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '20-29 FG'}, inplace=True)

cleanFieldGoalFrom20To29Yards2020Series = \
fieldGoalFrom20To29Yards2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '20-29 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom20To29Yards2020Frame = pd.DataFrame(cleanFieldGoalFrom20To29Yards2020Series)
fieldGoals2020RankingFor20To29Yards = cleanFieldGoalFrom20To29Yards2020Frame.reset_index()


# Field Goals Attempts from 20 to 19 Yards
onlyFieldGoalsAttemptFrom20To29Yards2020Series = onlyFieldGoalsAttemptFrom20To29Yards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom20To29Yards2020Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom20To29Yards2020Series)

onlyFieldGoalsAttemptFrom20To29Yards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '20-29 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom20To29Yards2020Series = \
onlyFieldGoalsAttemptFrom20To29Yards2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '20-29 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom20To29Yards2020Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom20To29Yards2020Series)
fieldGoalsAttempts2020RankingFor20To29Yards = cleanFieldGoalsAttemptFrom20To29Yards2020Frame.reset_index()


###############################################################################################################################################################


# Field Goals from 30 to 39 Yards
fieldGoalFrom30To39Yards2020Series = onlyFieldGoalsFrom30To39Yards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom30To39Yards2020Frame = pd.DataFrame(fieldGoalFrom30To39Yards2020Series)

fieldGoalFrom30To39Yards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '30-39 FG'}, inplace=True)

cleanFieldGoalFrom30To39Yards2020Series = \
fieldGoalFrom30To39Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '30-39 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom30To39Yards2020Frame = pd.DataFrame(cleanFieldGoalFrom30To39Yards2020Series)
fieldGoals2020RankingFor30To39Yards = cleanFieldGoalFrom30To39Yards2020Frame.reset_index()


# Field Goals Attempts from 30 to 39 Yards
onlyFieldGoalsAttemptFrom30To39Yards2020Series = onlyFieldGoalsAttemptFrom30To39Yards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom30To39Yards2020Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom30To39Yards2020Series)

onlyFieldGoalsAttemptFrom30To39Yards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '30-39 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom30To39Yards2020Series = \
onlyFieldGoalsAttemptFrom30To39Yards2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '30-39 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom30To39Yards2020Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom30To39Yards2020Series)
fieldGoalsAttempts2020RankingFor30To39Yards = cleanFieldGoalsAttemptFrom30To39Yards2020Frame.reset_index()


###############################################################################################################################################################


# Field Goals from 40 to 49 Yards
fieldGoalFrom40To49Yards2020Series = onlyFieldGoalsFrom40To49Yards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom40To49Yards2020Frame = pd.DataFrame(fieldGoalFrom40To49Yards2020Series)

fieldGoalFrom40To49Yards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '40-49 FG'}, inplace=True)

cleanFieldGoalFrom40To49Yards2020Series = \
fieldGoalFrom40To49Yards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '40-49 FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom40To49Yards2020Frame = pd.DataFrame(cleanFieldGoalFrom40To49Yards2020Series)
fieldGoals2020RankingFor40To49Yards = cleanFieldGoalFrom40To49Yards2020Frame.reset_index()


# Field Goals Attempts from 40 to 49 Yards
onlyFieldGoalsAttemptFrom40To49Yards2020Series = onlyFieldGoalsAttemptFrom40To49Yards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom40To49Yards2020Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom40To49Yards2020Series)

onlyFieldGoalsAttemptFrom40To49Yards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '40-49 FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom40To49Yards2020Series = \
onlyFieldGoalsAttemptFrom40To49Yards2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '40-49 FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom40To49Yards2020Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom40To49Yards2020Series)
fieldGoalsAttempts2020RankingFor40To49Yards = cleanFieldGoalsAttemptFrom40To49Yards2020Frame.reset_index()


###############################################################################################################################################################

# Field Goals from 50+ Yards
fieldGoalFrom50PlusYards2020Series = onlyFieldGoalsFrom50PlusYards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
fieldGoalFrom50PlusYards2020Frame = pd.DataFrame(fieldGoalFrom50PlusYards2020Series)

fieldGoalFrom50PlusYards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '50+ FG'}, inplace=True)

cleanFieldGoalFrom50PlusYards2020Series = \
fieldGoalFrom50PlusYards2019Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '50+ FG'].sum().sort_values(ascending=False)
cleanFieldGoalFrom50PlusYards2020Frame = pd.DataFrame(cleanFieldGoalFrom50PlusYards2020Series)
fieldGoals2020RankingFor50PlusYards = cleanFieldGoalFrom50PlusYards2020Frame.reset_index()


# Field Goals Attempts from 50+ Yards
onlyFieldGoalsAttemptFrom50PlusYards2020Series = onlyFieldGoalsAttemptFrom50PlusYards2020.groupby(
    ['season','possessionTeam', 'displayName', 'position'], as_index=False).size()
onlyFieldGoalsAttemptFrom50PlusYards2020Frame = pd.DataFrame(onlyFieldGoalsAttemptFrom50PlusYards2020Series)

onlyFieldGoalsAttemptFrom50PlusYards2020Frame.rename(
    columns={'season': 'Season', 'displayName': 'Player Name', 'possessionTeam': 'Team', 'position': 'Position',
             'size': '50+ FGA'}, inplace=True)

cleanFieldGoalsAttemptFrom50PlusYards2020Series = \
onlyFieldGoalsAttemptFrom50PlusYards2020Frame.groupby(['Season', 'Player Name', 'Team', 'Position'])[
    '50+ FGA'].sum().sort_values(ascending=False)
cleanFieldGoalsAttemptFrom50PlusYards2020Frame = pd.DataFrame(cleanFieldGoalsAttemptFrom50PlusYards2020Series)
fieldGoalsAttempts2020RankingFor50PlusYards = cleanFieldGoalsAttemptFrom50PlusYards2020Frame.reset_index()

In [276]:


###############################################################################################################################################################
mergeFiledGoalAndAttemptsFrom1To19Yards2020 = pd.merge(fieldGoals2020RankingFor1To19Yards, 
                                                   fieldGoalsAttempts2020RankingFor1To19Yards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom1To19Yards2020.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom1To19Yards2020['1-19 FG'] = mergeFiledGoalAndAttemptsFrom1To19Yards2020['1-19 FG'].astype(int)

mergeFiledGoalAndAttemptsFrom1To19Yards2020["1 - 19: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom1To19Yards2020["1-19 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom1To19Yards2020["1-19 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom1To19Yards2020.drop(['1-19 FG','1-19 FGA'],axis=1, inplace=True)


###############################################################################################################################################################


mergeFiledGoalAndAttemptsFrom20To29Yards2020 = pd.merge(fieldGoals2020RankingFor20To29Yards, 
                                                   fieldGoalsAttempts2020RankingFor20To29Yards,
                                                   how='right')

mergeFiledGoalAndAttemptsFrom20To29Yards2020.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom20To29Yards2020['20-29 FG'] = mergeFiledGoalAndAttemptsFrom20To29Yards2020['20-29 FG'].astype(int)



mergeFiledGoalAndAttemptsFrom20To29Yards2020["20 - 29: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom20To29Yards2020["20-29 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom20To29Yards2020["20-29 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom20To29Yards2020.drop(['20-29 FG','20-29 FGA'],axis=1, inplace=True)


###############################################################################################################################################################


mergeFiledGoalAndAttemptsFrom30To39Yards2020 = pd.merge(fieldGoals2020RankingFor30To39Yards, 
                                                   fieldGoalsAttempts2020RankingFor30To39Yards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom30To39Yards2020.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom30To39Yards2020['30-39 FG'] = mergeFiledGoalAndAttemptsFrom30To39Yards2020['30-39 FG'].astype(int)



mergeFiledGoalAndAttemptsFrom30To39Yards2020["30 - 39: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom30To39Yards2020["30-39 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom30To39Yards2020["30-39 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom30To39Yards2020.drop(['30-39 FG','30-39 FGA'],axis=1, inplace=True)


###############################################################################################################################################################

mergeFiledGoalAndAttemptsFrom40To49Yards2020 = pd.merge(fieldGoals2020RankingFor40To49Yards, 
                                                   fieldGoalsAttempts2020RankingFor40To49Yards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom40To49Yards2020.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom40To49Yards2020['40-49 FG'] = mergeFiledGoalAndAttemptsFrom40To49Yards2020['40-49 FG'].astype(int)



mergeFiledGoalAndAttemptsFrom40To49Yards2020["40 - 49: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom40To49Yards2020["40-49 FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom40To49Yards2020["40-49 FGA"].map(str)
mergeFiledGoalAndAttemptsFrom40To49Yards2020.drop(['40-49 FG','40-49 FGA'],axis=1, inplace=True)


###############################################################################################################################################################

mergeFiledGoalAndAttemptsFrom50PlusYards2020 = pd.merge(fieldGoals2020RankingFor50PlusYards, 
                                                   fieldGoalsAttempts2020RankingFor50PlusYards, 
                                                   how='right')

mergeFiledGoalAndAttemptsFrom50PlusYards2020.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttemptsFrom50PlusYards2020['50+ FG'] = mergeFiledGoalAndAttemptsFrom50PlusYards2020['50+ FG'].astype(int)



mergeFiledGoalAndAttemptsFrom50PlusYards2020["50+: FGM / FGA"] = mergeFiledGoalAndAttemptsFrom50PlusYards2020["50+ FG"].map(str)  + " / " + mergeFiledGoalAndAttemptsFrom50PlusYards2020["50+ FGA"].map(str)
mergeFiledGoalAndAttemptsFrom50PlusYards2020.drop(['50+ FG','50+ FGA'],axis=1, inplace=True)



In [277]:
mergeFiledGoalAndAttempts2020 = pd.merge(fieldGoals2020Ranking, 
                                     fieldGoalsAttempts2020Ranking['FG Attempts'], 
                                     left_index=True, 
                                     right_index=True, 
                                     how='outer'
                                    )

###############################################################################################################################################################

mergeFiledGoalAndAttempts2020['FG Accuracy %'] = (mergeFiledGoalAndAttempts2020['Field Goals'] / mergeFiledGoalAndAttempts2020['FG Attempts']).round(3) * 100

###############################################################################################################################################################

mergeFiledGoalAndAttempts2020.replace(np.nan, 0,inplace=True)
mergeFiledGoalAndAttempts2020['Field Goals'] = mergeFiledGoalAndAttempts2020['Field Goals'].astype(int)


In [278]:

addFGFor1To19Yards2020 = pd.merge(mergeFiledGoalAndAttempts2020, 
                              mergeFiledGoalAndAttemptsFrom1To19Yards2020, 
                              how='left'
                             )
###############################################################################################################################################################

addFGFor20To29Yards2020 = pd.merge(addFGFor1To19Yards2020, 
                               mergeFiledGoalAndAttemptsFrom20To29Yards2020, 
                               how='left'
                              )
###############################################################################################################################################################

addFGFor30To39Yards2020 = pd.merge(addFGFor20To29Yards2020, 
                               mergeFiledGoalAndAttemptsFrom30To39Yards2020, 
                               how='left'
                              )

###############################################################################################################################################################

addFGFor40To49Yards2020 = pd.merge(addFGFor30To39Yards2020, 
                               mergeFiledGoalAndAttemptsFrom40To49Yards2020, 
                               how='left'
                              )

###############################################################################################################################################################

addFGFor40To49Yards2020.replace(np.nan,'0 / 0', inplace = True)


In [279]:

mergeFiledGoalMissed2020 = pd.merge(addFGFor40To49Yards2020, 
                                fieldGoalsMissed2020Ranking, 
                                how='outer'
                               )

mergeFiledGoalMissed2020.replace(np.nan, 0, inplace=True)


###############################################################################################################################################################

mergeFieldGoalBlocked2020 = pd.merge(mergeFiledGoalMissed2020,
                                 fieldGoalsBlocked2020Ranking,
                                 how='outer'
                                )

initial2020FieldGoalStat = mergeFieldGoalBlocked2020.replace(np.nan, 0)

###############################################################################################################################################################

initial2020FieldGoalStat['Season'] = initial2020FieldGoalStat['Season'].astype(int)
initial2020FieldGoalStat['Field Goals'] = initial2020FieldGoalStat['Field Goals'].astype(int)
initial2020FieldGoalStat['FG Attempts'] = initial2020FieldGoalStat['FG Attempts'].astype(int)
initial2020FieldGoalStat['FG Missed'] = initial2020FieldGoalStat['FG Missed'].astype(int)
initial2020FieldGoalStat['FG Blocked'] = initial2020FieldGoalStat['FG Blocked'].astype(int)

In [280]:

final2020FieldGoalStat = initial2020FieldGoalStat[(initial2020FieldGoalStat.Season != 0)]

FieldGoal2020 = final2020FieldGoalStat.rename(
    columns={'Field Goals': 'FG Made', '1 - 19: FGM / FGA': '1-19 yds: FGM/FGA', '20 - 29: FGM / FGA': '20-29 yds: FGM/FGA', '30 - 39: FGM / FGA': '30-39 yds: FGM/FGA',
             '40 - 49: FGM / FGA': '40-49 yds: FGM/FGA'})


In [281]:
FieldGoal2020

,Season,Player Name,Team,Position,FG Made,FG Attempts,FG Accuracy %,1-19 yds: FGM/FGA,20-29 yds: FGM/FGA,30-39 yds: FGM/FGA,40-49 yds: FGM/FGA,FG Missed,FG Blocked
0,2020,Jason Sanders,MIA,K,32,42,76.2,13 / 13,8 / 10,0 / 12,0 / 0,3,0
1,2020,Younghoe Koo,ATL,K,31,41,75.6,18 / 18,6 / 7,0 / 8,0 / 0,2,0
2,2020,Rodrigo Blankenship,IND,K,31,38,81.6,18 / 19,12 / 13,0 / 4,0 / 0,5,0
3,2020,Daniel Carlson,LV,K,31,38,81.6,22 / 22,5 / 7,0 / 4,0 / 0,2,0
4,2020,Greg Zuerlein,DAL,K,30,36,83.3,13 / 13,11 / 12,0 / 8,0 / 3,6,0
5,2020,Graham Gano,NYG,K,29,35,82.9,12 / 12,10 / 10,0 / 8,0 / 0,1,0
6,2020,Cairo Santos,CHI,K,27,35,77.1,15 / 15,9 / 10,0 / 4,0 / 0,2,0
7,2020,Brandon McManus,DEN,K,27,34,79.4,10 / 11,7 / 7,0 / 12,0 / 3,5,2
8,2020,Ryan Succop,TB,K,26,33,78.8,15 / 15,9 / 11,0 / 3,0 / 0,2,1
9,2020,Ka'imi Fairbairn,HOU,K,25,33,75.8,14 / 14,8 / 10,0 / 4,0 / 0,3,0


In [282]:
fieldGoalFull = pd.concat([FieldGoal2018, FieldGoal2019, FieldGoal2020], axis=0)
fieldGoalFull.reset_index(drop=True, inplace=True)

In [283]:
fieldGoalFull.to_csv(r'/Users/kushtrivedi/Desktop/NFL/Field_Goals_2018_to_2020.csv', index=False)

In [284]:
ffs = fieldGoalFull.groupby(['Team','Player Name','Season','FG Accuracy %','FG Made','FG Missed','FG Blocked','1-19 yds: FGM/FGA',
                             '20-29 yds: FGM/FGA','30-39 yds: FGM/FGA','40-49 yds: FGM/FGA'], 
                             as_index=True).size()           
                             

In [285]:
ffss = pd.DataFrame(ffs)

In [286]:
pd.options.display.max_rows = 99999999
ffss

0
Team Player Name          Season FG Accuracy % FG Made FG Missed FG Blocked 1-19 yds: FGM/FGA 20-29 yds: FGM/FGA 30-39 yds: FGM/FGA 40-49 yds: FGM/FGA   
ARI  Mike Nugent          2020   66.7          6       0         1          4 / 5             2 / 2              0 / 0              0 / 0               1
     Phil Dawson          2018   71.4          5       3         0          4 / 4             1 / 3              0 / 1              0 / 0               1
     Zane Gonzalez        2018   50.0          6       2         0          3 / 3             1 / 1              2 / 4              0 / 0               1
                          2019   80.0          28      2         1          21 / 22           4 / 5              3 / 4              0 / 0               1
                          2020   75.0          15      6         0          6 / 6             4 / 6              0 / 9              0 / 0               1
ATL  Elliott Fry          2020   50.0          1       0         0          1 / 1             0 / 0              0 / 0              0 / 0               1
     Giorgio Tavecchio    2018   83.3          5       0         0          1 / 1             2 / 2              2 / 2              0 / 0               1
     Matt Bryant          2018   71.4          20      1         0          8 / 8             7 / 7              5 / 6              0 / 0               1
                          2019   66.7          8       5         0          4 / 5             3 / 3              1 / 5              0 / 0               1
     Younghoe Koo         2019   67.7          21      3         0          13 / 13           6 / 9              2 / 2              0 / 0               1
                          2020   75.6          31      2         0          18 / 18           6 / 7              0 / 8              0 / 0               1
BAL  Justin Tucker        2018   85.4          35      0         2          19 / 19           10 / 11            6 / 7              0 / 1               1
                          2019   80.0          28      1         0          15 / 15           9 / 10             4 / 4              0 / 0               1
                          2020   77.4          24      3         0          10 / 11           10 / 9             0 / 5              0 / 1               1
BUF  Stephen Hauschka     2018   72.4          21      4         1          11 / 11           6 / 8              4 / 7              0 / 0               1
                          2019   68.8          22      5         0          10 / 11           11 / 11            1 / 4              0 / 1               1
     Tyler Bass           2020   71.0          22      6         0          10 / 12           6 / 8              0 / 6              0 / 2               1
CAR  Chandler Catanzaro   2018   71.4          5       0         0          3 / 3             0 / 0              2 / 2              0 / 0               1
     Graham Gano          2018   76.5          13      2         0          6 / 7             5 / 5              1 / 2              1 / 1               1
     Joey Slye            2019   68.8          22      5         1          8 / 9             4 / 6              10 / 13            0 / 0               1
                          2020   71.4          20      6         1          11 / 12           7 / 8              0 / 5              0 / 2               1
CHI  Cairo Santos         2020   77.1          27      2         0          15 / 15           9 / 10             0 / 4              0 / 0               1
     Cody Parkey          2018   76.7          23      7         0          12 / 14           9 / 13             2 / 3              0 / 0               1
     Eddy Pineiro         2019   67.9          19      4         0          12 / 14           5 / 7              2 / 3              0 / 0               1
CIN  Austin Seibert       2020   71.4          5       2         0          4 / 4             0 / 0              0 / 3              0 / 0           